# 11. YÜZ ÖZELLİKLERİNİ KULLANMA (USİNG FACE FEATURE) 1.ADIM 12.YÜZ ÖZELLİKLERİNİ KULLANMA 2.ADIM VE 13. YÜZ ÖZELLİKLERİNİ KULLANMA 3. ADIM

In [ ]:
import cv2
import numpy as np
import math

def findMaxContour(contours): #findMaxContour adında bir fonksiyon tanımladık ve bu fonksiyon bize contourların maksimum değerlerini verecek
    max_i = 0  # maksimum i değerini şimdilik 0 olarak belirledik çünkü ilerleyen kısımlarda i değeri kendi kendine artacak yani klasik for döngüsü mantığı
    max_area = 0 # maksimum i değeri 0 olduğu için maksimum alanda 0 olacaktır
    
    for i in range(len(contours)): # kontur değerlerinin uzunluğu listesi içinde i değeri dönecektir
        area_face = cv2.contourArea(contours[i]) # kontürlerde alan bulmak için cv2.contourArea(contours) yazılır ama biz belirli bir i değerinin dolanması için contours[i] şekilnde yazdık
        if max_area < area_face: # eğer maksimum alan yüzümüzün alanından küçükse 
            max_area = area_face # maksimum alan yüzün alanıdır
            max_i = i # ve doğal olarak maksimum alanı yani bu karar yapısı için yüzümüzün alanını veren i değeri maksimum i değeri oluyor
            
        try:  # kodumuz çalışırken bir hata vermesin diye contours içinde maximum i değerini gezdiriyoruz
            c = contours[max_i]
            
        except: # eğer try çalışmazsa bunu yapacaktır
            contours = [0] # 
            c = contours[0] # 
            
        return c # c değerini döndürüyoruz 
        
    
cap = cv2.VideoCapture(0) # videomuzu alıyoruz

while True:
    ret,frame = cap.read() #frameleri döndürüyoruz
    frame = cv2.flip(frame,1) #sağ elimizi kaldırdığımızda ekranın bize göre sağ tarafındaki el kalksın diye flip ediyoruz
    if ret is False: # eğer ret değişkeni frame alamazsa framelerle iş yapan fonksiyonlarımız sıkıntıya giriyor bu sıkıntı olmasın diye ret değişkeni frame almadığında işlemi bitiriyoruz ve pencerelerimiz kapanıyor
        break
    
    roi = frame[40:280,150:450] #frame[y1:y2,x1:x2] şeklindedir 
    cv2.rectangle(frame,(150,40),(450,280),(0,0,250),0) # cv2.rectangle içine 4 değer alır frame,(x1,y1),(x2,y2),(renk değeri),kalınlık değeri dir bu değerler
                                                        # bu rectangle ın kalınlık değeri 0 olmak zorundadır çünkü kalınlığı artınca threshold'a maruz kalıp maskelenebilir
    hsv = cv2.cvtColor(roi,cv2.COLOR_BGR2HSV) # bgr uzayında hsv uzayına dönüşüm yapıyoruz
    lower_color = np.array([0,45,79],dtype=np.uint8) # ortalama ten rengi alt değerlerini bir array içine alıyoruz
    upper_color = np.array([17,255,255],dtype=np.uint8) # ortalama ten rengi üst değerlerini bir sıra içine alıyoruz
    
    mask = cv2.inRange(hsv,lower_color,upper_color) # cv2.inRange fonksiyonu maskeleme işlemi yapar, ilk aldığı değer maskelemnin yapılacağı dönüştürülmüş renk uzayı, ikinci değer alt değerler,üçüncü değer üst değerlerdir
    
    kernel = np.ones((3,3),dtype=np.uint8) # 3 e 3 lük bir kernel oluşturduk çünkü
    mask = cv2.dilate(mask,kernel,iterations=7) # cv2.dilate içine 3 değer alır birincisi mask, ikincisi kernel değeri, üçüncüsü iterasyon yani yineleme değeridir
    mask = cv2.medianBlur(mask,15) # görüntüdeki gürültüler daha da azalsın diye medianBlur uyguladık
    
    contours,ret = cv2.findContours(mask,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) # kontür bulma fonksiyonudur ilk değer mask, ikinci ve üçüncü değer default değerlerdir
    
    if len(contours) > 0: #eğer kontürlerin uzunluğu 0 dan büyükse diyoruz çünkü 0 a eşitse işlem yapamayız ve sıkıntı olur
        
            c = findMaxContour(contours) # ilk satırlarda belirlediğimiz findMaxContours fonksiyonunu burada kullanıyoruz 
            
            ensol = tuple(c[c[:, :, 0].argmin()][0])  # bu ifade argmin() yardımıyla bize tüm contour değerlerini dolanarak  değerlerin 1.indisini yani x değerini en küçüklerini bulacak
            ensag = tuple(c[c[:, :, 0].argmax()][0]) # bu ifadede argmax() bize tüm contour değerlerini dolanarak  değerlerin 1.indisini yani x değerini en büyüklerini bulacak
            entepe = tuple(c[c[:, :, 1].argmin()][0])  # tepe noktası y ekseninde olduğu için contour değerlerindeki 2. indisi aldık
            endip = tuple(c[c[:, :, 1].argmax()][0])
            
            cv2.circle(roi,ensag,5,(0,255,0),2) # circle oluşturmak için gerekli değerleri girdik
            cv2.circle(roi,ensol,5,(0,255,0),2)
            cv2.circle(roi,entepe,5,(0,255,0),2)
            cv2.circle(roi,endip,5,(0,255,0),2)
            
            cv2.line(roi,ensol,entepe,(255,0,0),2) # lineları çizerken bir değerin bittiği yerden başlamak gerekli yoksa sıkıntı düzgün çalışmıyor 
            cv2.line(roi,entepe,ensag,(255,0,0),2)
            cv2.line(roi,ensag,endip,(255,0,0),2)
            cv2.line(roi,endip,ensol,(255,0,0),2)
            
            a = math.sqrt((ensag[0]-entepe[0])**2+(ensag[1]-entepe[1])**2) # burada 2 nokta arasındaki uzaklık formülünü kullanarak a b c değerlerini bulduk
            b = math.sqrt((endip[0]-ensag[0])**2+(endip[1]-ensag[1])**2)
            c = math.sqrt((ensag[0]-ensol[0])**2+(ensag[1]-ensol[1])**2) # en uzun kenarı c olan geniş açılı bir üçgen düşün işte oradaki c bu
            
            try: # bu sefer try ve except karar yapılarını yazma sebebimiz farklı,burada hem sayı/0 yani division by zero gelmesin diye hem de eğer olurda açı bulamazsak diye yazıyoruz
                angle_ab = int(math.acos((a**2+b**2-c**2)/(2*b*c))*57)
                cv2.putText(roi,str(angle_ab),(ensag[0]-100+50,ensag[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA) #cv2.putText() içine 8 değer alıyor ilk değer görselimiz, ikinci değer yazımız, üçüncü değer font, 4. değer fontün büyüklüğü, 6. değer renk, 7. değer yazının kalınlığı 8. değer default  değer
            except:
                cv2.putText(roi,"?",(ensag[0]-100+50,ensag[1]),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA) 
                
    cv2.imshow("frame",frame)
    cv2.imshow("roi",roi)
    cv2.imshow("mask",mask)
    
    if cv2.waitKey(5) == ord('q'):  # q tuşuna basıldığında işlemleri bitirir
        break
        
cap.release()
cv2.destroyAllWindows()